# March Madness Predictions

Jason Lott, Brendan LaPuma, Eamon Weingold

In [1]:
import pandas as pd, numpy as np
import tensorflow as tf

2023-03-13 01:31:57.733543: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 01:31:57.906261: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-13 01:31:57.906292: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-13 01:31:57.938166: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-13 01:32:00.033605: W tensorflow/stream_executor/pla

In [2]:
from tensorflow import keras
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

Import kenpom stats for every team from years 2002-2017

In [3]:
kenpom_df = pd.read_csv("data/kenpom_2017.csv")

Import tourney game results for every tourney game from years 2002-2017

In [4]:
tgr_df = pd.read_csv("data/MNCAATourneyCompactResults.csv")
tgr_df = tgr_df.loc[tgr_df["Season"] >= 2002]
tgr_df = tgr_df.loc[tgr_df["Season"] <= 2017]

Import kaggle IDs for each team

In [5]:
IDs_df = pd.read_csv("data/MTeams.csv")
IDs_df = IDs_df.iloc[:, np.r_[0,1]]

Change team names in kenpom and tgr to thier kaggle IDs

In [6]:
namechange_dict = {'Cal St.': 'CS',
'Albany': 'SUNY Albany',
'Abilene Christian': 'Abilene Chr',
'American': 'American Univ',
'Arkansas Little Rock': 'Ark Little Rock',
'Arkansas Pine Bluff': 'Ark Pine Bluff',
'Boston University': 'Boston Univ',
'Central Michigan': 'C Michigan',
'Central Connecticut': 'Central Conn',
'Coastal Carolina': 'Coastal Car',
'Eastern Washington': 'E Washington',
'East Tennessee St': 'ETSU',
'Fairleigh Dickinson': 'F Dickinson',
'Florida Atlantic': 'FL Atlantic',
'Florida Gulf Coast': 'FL Gulf Coast',
'George Washington': 'G Washington',
'Illinois Chicago': 'IL Chicago',
'Kent St': 'Kent',
'Monmouth': 'Monmouth NJ',
'Mississippi Valley St': 'MS Valley St',
'Northern Colorado': 'N Colorado',
'North Dakota St': 'N Dakota St',
'Northern Kentucky': 'N Kentucky',
'North Carolina A&T': 'NC A&T',
'North Carolina Central': 'NC Central',
'North Carolina St': 'NC State',
'Northwestern St': 'Northwestern LA',
'Prairie View A&M': 'Prairie View',
'South Carolina St': 'S Carolina St',
'South Dakota St': 'S Dakota St',
'Southern Illinois': 'S Illinois',
'Southeastern Louisiana': 'SE Louisiana',
'Stephen F Austin': 'SF Austin',
'Southern': 'Southern Univ',
'Southern Univ Miss': 'Southern Miss',
'Saint Louis': 'St Louis',
'Texas A&M Corpus Chris': 'TAM C. Christi',
'Troy St': 'Troy',
'Texas Southern Univ': 'TX Southern',
'Louisiana Lafayette': 'Louisiana',
'UTSA': 'UT San Antonio',
'Western Michigan': 'W Michigan',
'Green Bay': 'WI Green Bay',
'Milwaukee': 'WI Milwaukee',
'Western Kentucky': 'WKU',
'College of Charleston': 'Col Charleston',
'Loyola Chicago': 'Loyola-Chicago',
'The Citadel':'Citadel',
'UMKC':"Missouri KC",
'Northern Illinois':'N Illinois',
"Louisiana Monroe":'ULM',
'Middle Tennessee':'MTSU',
'Middle Tennessee St':'MTSU',
'Western Carolina':'W Carolina',
'Eastern Illinois': 'E Illinois',
'FIU':'Florida Intl',
'Tennessee Martin':'TN Martin',
'Western Illinois': 'W Illinois',
'Stephan F Austin':"SF Austin",
'Loyola Marymount':'Loy Marymount',
'Charleston Southern':'Charleston So',
'Bethune Cookman':'Bethune-Cookman',
'Eastern Kentucky':'E Kentucky',
'Maryland Eastern Shore':'MD E Shore',
'Birmingham Southern':'Birmingham So',
'Eastern Michigan':'E Michigan',
'Texas Southern':'TX Southern',
'Georgia Southern': 'Ga Southern',
'Cal St Northridge':'CS Northridge',
'Saint Mary\'s': 'St Mary\'s CA',
'Cal St Fullerton':'CS Fullerton',
'Saint Joseph\'s':'St Joseph\'s PA',
'Southwest Texas St':'Texas St',
'Southeast Missouri St':'SE Missouri St',
'Sacramento St':'CS Sacramento',
'Southwest Missouri St':'Missouri St',
'Texas Pan American':'UTRGV',
'UT Rio Grande Valley':'UTRGV',
'IPFW':'PFW',
'Fort Wayne':'PFW',
'Saint Peter\'s':'St Peter\'s',
'Grambling St':'Grambling',
'Utah Valley St':'Utah Valley',
'Mount St Mary\'s':'Mt St Mary\'s',
'Kennesaw St':'Kennesaw',
'Central Arkansas':'Cent Arkansas',
'Winston Salem St':'W Salem St',
'Cal St Bakersfield':'CS Bakersfield',
'USC Upstate':'SC Upstate',
'SIU Edwardsville':'SIUE',
'Houston Baptist':'Houston Chr',
'Nebraska Omaha':'NE Omaha',
'UMass Lowell':'MA Lowell',
'Little Rock':'Ark Little Rock'}

In [7]:
def refactor_teamnames(team_column):
    teamnames = pd.Series(team_column)
    teamnames = teamnames.str.strip()
    teamnames = teamnames.str.replace('.', '', regex=False)
    teamnames = teamnames.str.replace('\',', '\'', regex=False)
    teamnames = teamnames.replace(namechange_dict)
    
    return teamnames

In [8]:
kenpom_df['Team'] = refactor_teamnames(kenpom_df['Team'])

kenpom_df['Team'] = kenpom_df['Team'].str.lower()

# load the csv file with team names and IDs into a dataframe
teams_df = pd.read_csv('data/MTeamSpellings.csv')

# merge the two dataframes based on the team name column
merged_df = pd.merge(kenpom_df, teams_df, how='left', left_on='Team', right_on='TeamNameSpelling')

# replace the team name column with the ID column
kenpom_df['Team Name'] = kenpom_df['Team']
kenpom_df['Team'] = merged_df['TeamID']


In [9]:
kenpom_df["Team"] = kenpom_df["Team"].astype(int)
kenpom_df['Power5'] = kenpom_df['Conference'].isin(['ACC', 'B10', 'SEC', 'B12', 'P10', 'P12']).astype(int)
kenpom_df["WPct"] = kenpom_df["Wins"] / (kenpom_df["Wins"] + kenpom_df["Losses"])
kenpom_df = kenpom_df.drop(columns=["Conference", "Wins", "Losses", "AdjustO Rank", "AdjustD Rank","AdjustT Rank","Luck Rank","SOS Pyth Rank","SOS OppO Rank","SOS OppD Rank","NCSOS Pyth Rank"])

In [10]:
i = 0
for year in kenpom_df.groupby("Year"):
    year_data = year[1]
    for col in ["Pyth","AdjustO","AdjustD","AdjustT","Luck","SOS Pyth","SOS OppO","SOS OppD","NCSOS Pyth", "WPct"]:
        year_data[col] = (year_data[col] - year_data[col].min()) / (year_data[col].max() - year_data[col].min())
    if i == 0:
        normal_kenpom_df = year_data.copy()
        i += 1
    else:
        normal_kenpom_df = pd.concat([normal_kenpom_df, year_data],ignore_index=True,sort=False)

In [11]:
tgr_df2 = tgr_df.copy()
tgr_df2["Result"] = np.select([tgr_df["WTeamID"] > tgr_df["LTeamID"], tgr_df["WTeamID"] < tgr_df["LTeamID"]], [1, 0])
idx = (tgr_df2["Result"] == 0)
tgr_df2.loc[idx,["WTeamID", "WScore", "LTeamID", "LScore"]] = tgr_df2.loc[idx,["LTeamID", "LScore", "WTeamID", "WScore"]].values
tgr_df2 = tgr_df2.rename(columns={"WTeamID":"HTeamID", "WScore":"HScore"})
tgr_df2 = tgr_df2.drop(columns=["DayNum", "NumOT", "WLoc", "Result"])

Below is the training data table. Contains every tourney game 2002-2017 w/ corresponding kenpom data from that year, and result of that game

In [12]:
tgr_w_kenpom_df = pd.merge(tgr_df2, normal_kenpom_df, how='left', left_on=['Season','HTeamID'], right_on=['Year','Team'])
tgr_w_kenpom_df = pd.merge(tgr_w_kenpom_df, normal_kenpom_df, how='left', left_on=['Season','LTeamID'], right_on=['Year','Team'])
tgr_w_kenpom_df = tgr_w_kenpom_df.drop(columns=['Year_x','Year_y', 'Team_x', 'Team_y'])
cols = ["Rank","Seed","Pyth","AdjustO","AdjustD","AdjustT","Luck","SOS Pyth","SOS OppO","SOS OppD","NCSOS Pyth", "Power5","WPct"]
tgr_w_kenpom_df

,Season,HTeamID,HScore,LTeamID,LScore,Rank_x,Seed_x,Pyth_x,AdjustO_x,AdjustD_x,...,AdjustD_y,AdjustT_y,Luck_y,SOS Pyth_y,SOS OppO_y,SOS OppD_y,NCSOS Pyth_y,Team Name_y,Power5_y,WPct_y
0,2002,1373,81,1108,77,174,16.0,0.458219,0.529138,0.428169,...,0.619718,0.758065,0.842105,0.125076,0.133690,0.884354,0.574879,alcorn st,0,0.736534
1,2002,1194,78,1104,86,191,15.0,0.438476,0.501166,0.430986,...,0.239437,0.556452,0.768421,0.828667,0.780749,0.136054,0.456664,alabama,1,0.859083
2,2002,1364,81,1112,86,77,14.0,0.612489,0.701632,0.352113,...,0.338028,0.681452,0.778947,1.000000,1.000000,0.034014,1.000000,arizona,1,0.782408
3,2002,1457,37,1181,84,219,16.0,0.395470,0.447552,0.445070,...,0.000000,0.818548,0.407018,0.867620,0.882353,0.183673,0.690253,duke,1,0.992774
4,2002,1428,56,1231,75,45,12.0,0.692225,0.776224,0.295775,...,0.067606,0.459677,0.329825,0.996957,0.978610,0.006803,0.885763,indiana,1,0.747073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040,2017,1314,75,1246,73,3,1.0,0.938612,0.868486,0.168478,...,0.138587,0.587156,0.586093,0.875857,0.857143,0.143939,0.553841,kentucky,1,0.878815
1041,2017,1376,77,1196,70,24,7.0,0.819683,0.575682,0.048913,...,0.086957,0.449541,0.430464,0.929267,0.844720,0.015152,0.713568,florida,1,0.774121
1042,2017,1376,73,1211,77,24,7.0,0.819683,0.575682,0.048913,...,0.000000,0.500000,0.632450,0.587514,0.590062,0.431818,0.455851,gonzaga,0,1.000000
1043,2017,1332,76,1314,77,10,3.0,0.887322,0.806452,0.187500,...,0.168478,0.555046,0.688742,0.930350,0.950311,0.143939,0.558507,north carolina,1,0.859372


Below will ignore the tournament seed in the training, add back in when available

In [13]:
train_df = tgr_w_kenpom_df.sample(frac = 0.8, random_state = 0)
test_df = tgr_w_kenpom_df.drop(train_df.index)

# remove below 2 lines to include tourney seed again
train_df = train_df.drop(columns = ["Seed_x", "Seed_y"])
test_df = test_df.drop(columns = ["Seed_x", "Seed_y"])

train_df2 = train_df.drop(columns = ["Rank_x", "Rank_y", "Team Name_x", "Team Name_y"])
test_df2 = test_df.drop(columns = ["Rank_x", "Rank_y", "Team Name_x", "Team Name_y"])

# will have to adjust columns selected when using tourney seed
x_train = train_df2.iloc[:, 5:29]
y_train = train_df2[["HScore", "LScore"]]

x_test = test_df2.iloc[:, 5:29]
y_test = test_df2[["HScore", "LScore"]] 

#x_train

Current best model - DO NOT CHANGE UNLESS FOUND A CONSISTENTLY BETTER ONE

In [14]:
# change input dim when including tourney seed
model = Sequential([
    Dense(64, activation="relu", input_dim=22),
    Dense(32, activation='relu'),
    Dense(2)
])

model.compile(loss="mean_squared_error", optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test))



Epoch 1/100
2023-03-13 01:32:04.037442: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-13 01:32:04.037483: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-13 01:32:04.037504: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-76c1b032-5d0a-4d37-bf01-322f3e813bd4): /proc/driver/nvidia/version does not exist
2023-03-13 01:32:04.037815: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
27/27 [==============================] - 1s 12ms/step - loss: 49

experiment model

Results of testing the model below

In [23]:
y_pred = model.predict(x_test)

y_pred = np.array(y_pred)
y_test = np.array(y_test)

y_pred = pd.DataFrame(y_pred, columns = ["HScore", "LScore"])
y_test = pd.DataFrame(y_test, columns = ["HScore", "LScore"])
sillylittlearray = np.column_stack((test_df2["Season"],test_df2["HTeamID"], test_df["Team Name_x"], test_df2["LTeamID"], test_df["Team Name_y"], test_df["Rank_x"], test_df["Rank_y"], y_pred["HScore"], y_pred["LScore"], y_test["HScore"], y_test["LScore"]))
pd.DataFrame(sillylittlearray, columns = ["Season","HTeamID", "HTeam Name", "LTeamID", "LTeam Name", "HTeamRank", "LTeamRank", "HScore Prediction", "LScore Prediction", "Actual HScore", "Actual LScore"])


7/7 [==============================] - 0s 3ms/step


,Season,HTeamID,HTeam Name,LTeamID,LTeam Name,HTeamRank,LTeamRank,HScore Prediction,LScore Prediction,Actual HScore,Actual LScore
0,2002,1332,oregon,1285,montana,9,210,84.683502,66.097374,81,62
1,2002,1214,hampton,1163,connecticut,116,18,65.1894,80.546127,67,78
2,2002,1373,siena,1268,maryland,174,3,60.45483,79.502991,70,85
3,2002,1301,nc state,1277,michigan st,21,32,69.455688,68.879326,69,58
4,2002,1403,texas tech,1356,s illinois,31,49,75.774376,74.219315,68,76
...,...,...,...,...,...,...,...,...,...,...,...
204,2017,1455,wichita st,1246,kentucky,8,4,66.021164,71.032845,62,65
205,2017,1276,michigan,1257,louisville,20,9,67.549866,69.586349,73,69
206,2017,1332,oregon,1276,michigan,10,20,69.524849,68.856354,69,68
207,2017,1462,xavier,1112,arizona,31,18,67.861008,68.484184,73,71


In [0]:
temp = [[1101,1102]]
for i in range(1101, 1478):
    for j in range(i+1, 1478):
        temp.append([i,j])
temp.remove([1101,1102])
submission_df = pd.DataFrame(temp, columns=["LTeamID","HTeamID"])
lteams = submission_df.LTeamID.values.tolist()
hteams = submission_df.HTeamID.values.tolist()
IDs = []
for i in range(0,70876):
    IDs.append("2023_" + str(lteams[i]) + "_" + str(hteams[i]))
submission_df["ID"] = IDs

import requests
from bs4 import BeautifulSoup
import re

def record_to_percent(s):
    n1, n2 = re.findall('(\\d+)-(\\d+)', s)[0]
    n1, n2 = float(n1), float(n2)
    return n1 / (n1 + n2)

bs = BeautifulSoup(open('kp2023.html'), 'html.parser')
bs.prettify()
df = pd.read_html(str(bs.find('table')))[0]
kenpom2023_df = df.iloc[:, np.r_[0:6, 7, 9, 11, 13, 15, 17, 19]]
kenpom2023_df = kenpom2023_df.copy()
kenpom2023_df.columns = ['Rankk','Team','Conference','W-L','AdjEM','AdjO','AdjD','AdjT','Luck','SOS AdjEM','SOS OppO','SOS OppD','NCSOS']

kenpom2023_df['Seed'] = kenpom2023_df['Team'].str.extract('( \d{1,2})', expand=True)
kenpom2023_df['Seed'] = kenpom2023_df['Seed'].str.strip()
kenpom2023_df['Seed'] = kenpom2023_df['Seed'].astype(float).astype('Int32')

kenpom2023_df['Team'] = kenpom2023_df['Team'].replace(to_replace=r' \d{1,2}', value='', regex=True)

kenpom2023_df

In [0]:
kenpom2023_df

In [0]:
# load the existing CSV file into a DataFrame and add new rows
altnames_df = pd.concat(
    [pd.read_csv('data/MTeamSpellings.csv'),
    pd.DataFrame([('st thomas', 1472), ('queens', 1474), ('texas a&m commerce', 1477), ('houston christian', 1223), ('liu', 1254)], columns=['TeamNameSpelling', 'TeamID'])]
)

# needed for shit to work, idk why
kenpom2023_df = kenpom2023_df.copy()

kenpom2023_df['Team'] = refactor_teamnames(kenpom2023_df['Team'])

kenpom2023_df['Team'] = kenpom2023_df['Team'].str.lower()

altnames_df

In [0]:
kenpom2023_df = kenpom2023_df.copy()

# merge the two dataframes based on the team name column
merged_df = pd.merge(kenpom2023_df, altnames_df, how='left', left_on='Team', right_on='TeamNameSpelling')

# replace the team name column with the ID column
kenpom2023_df['Team'] = merged_df['TeamID']

kenpom2023_df['team name'] = merged_df['TeamNameSpelling']
kenpom2023_df

In [0]:
#kenpom2023_df = pd.merge(kenpom2023_df, IDs_df, left_on="Team", right_on="TeamName", how="left")
#kenpom2023_df = kenpom2023_df.dropna()
#kenpom2023_df["Team"] = kenpom2023_df["Team ID"]
kenpom2023_df["Team"] = kenpom2023_df["Team"].astype(int)
kenpom2023_df['Power5'] = kenpom2023_df['Conference'].isin(['ACC', 'B10', 'SEC', 'B12', 'P10', 'P12']).astype(int)
kenpom2023_df['WPct'] = kenpom2023_df['W-L'].apply(lambda x: record_to_percent(x))
kenpom2023_df = kenpom2023_df.drop(columns=["Conference", "W-L"])

cols = ["AdjEM","AdjO","AdjD","AdjT","Luck","SOS AdjEM","SOS OppO","SOS OppD","NCSOS", "WPct"]
i = 0
for col in cols:
    kenpom2023_df[col] = (kenpom2023_df[col] - kenpom2023_df[col].min()) / (kenpom2023_df[col].max() - kenpom2023_df[col].min())
if i == 0:
    normal_kenpom2023_df = kenpom2023_df.copy()
    i += 1
else:
    normal_kenpom2023_df = pd.concat([normal_kenpom2023_df, kenpom2023_df],ignore_index=True,sort=False)


submission_df = pd.merge(submission_df, normal_kenpom2023_df, left_on="HTeamID", right_on="Team", how="left")
submission_df = pd.merge(submission_df, normal_kenpom2023_df, left_on="LTeamID", right_on="Team", how="left")
submission_df.drop(columns=["Team_x", "Team_y"], inplace=True)

submission_df

In [0]:
IDs = submission_df["ID"]
y_pred = model.predict(submission_df.drop(columns = ["HTeamID", "LTeamID", "Rankk_x", "Rankk_y", "ID", "team name_x", "team name_y", "Seed_x", "Seed_y"]))

sillylittlearray = np.column_stack((submission_df["HTeamID"], submission_df["LTeamID"], submission_df["Rankk_x"], submission_df["Rankk_y"], y_pred))

predictions_df = pd.DataFrame(sillylittlearray, columns = ["HTeamID", "LTeamID", "H kp rank", "L kp rank","Predictions"])

predictions_df.dropna(inplace=True)

# create a dictionary mapping IDs to team names using the first dataframe
id_to_team = dict(zip(altnames_df['TeamID'], altnames_df['TeamNameSpelling']))

# apply the dictionary to the second dataframe
predictions_df['Team Names (high + low'] = predictions_df['HTeamID'].map(id_to_team) + " " + predictions_df['LTeamID'].map(id_to_team)

predictions_df

In [0]:
bracketpredictor_df = predictions_df.drop(columns=["HTeamID", "LTeamID", "H kp rank", "L kp rank"])

bracketpredictor_df.to_csv("bracketpredictor.csv", index=False)

In [0]:
MPreds_df = predictions_df.copy()
MPreds_df["Predictions"] = MPreds_df["Predictions"].apply(lambda x: 1 - x)
MPreds_df.rename(columns={"Predictions":"Pred"},inplace=True)
MPreds_df.drop(columns=["HTeamID","LTeamID","H kp rank","L kp rank"], inplace=True)
MPreds_df["ID"] = IDs
cols = MPreds_df.columns.to_list()
cols = cols[-1:] + cols[:-1]
MPreds_df = MPreds_df[cols]
MPreds_df

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=76c1b032-5d0a-4d37-bf01-322f3e813bd4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>